In [ ]:
import pandas as pd
import json
from google.colab import files
from datetime import datetime
import numpy as np
from IPython.display import display, Markdown

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer)):
            return int(obj)
        if isinstance(obj, (np.floating)):
            return float(obj)
        if isinstance(obj, (datetime, pd.Timestamp)):
            return obj.isoformat()
        return super().default(obj)

def load_and_clean_data():
    """Handles file upload and data cleaning"""
    print("Please upload your Excel file:")
    try:
        uploaded = files.upload()
        if not uploaded:
            raise ValueError("No file uploaded.")
        filename = list(uploaded.keys())[0]
        df = pd.read_excel(filename, header=3)
    except Exception as e:
        print(f"Error uploading or reading file: {e}")
        return None, None

    # Validate required columns
    required_columns = ['EVENT DAY', 'RESOLVED TIME', 'SOURCE NAME', 'NAME', 'PATH']
    missing_cols = [col for col in required_columns if col not in df.columns]
    if missing_cols:
        print(f"Missing required columns: {', '.join(missing_cols)}")
        return None, None

    # Data cleaning
    df['EVENT DAY'] = pd.to_datetime(df['EVENT DAY'], format='%d %b %Y %H:%M:%S', errors='coerce', dayfirst=True)
    df['RESOLVED TIME'] = pd.to_datetime(df['RESOLVED TIME'], errors='coerce')
    df['CLIENT NAME'] = df.get('CLIENT NAME', '').replace({'americanauae': 'UAE', 'americanaksa': 'KSA'})
    df['SOURCE NAME'] = df['SOURCE NAME'].str.strip()
    df['resolution_minutes'] = (df['RESOLVED TIME'] - df['EVENT DAY']).dt.total_seconds() / 60
    df['month'] = df['EVENT DAY'].dt.strftime('%B')
    df['week_of_month'] = (df['EVENT DAY'].dt.day - 1) // 7 + 1
    df['week'] = df['EVENT DAY'].dt.isocalendar().week
    df['day'] = df['EVENT DAY'].dt.date

    # Filter valid resolutions
    valid_resolutions = df.dropna(subset=['RESOLVED TIME'])
    valid_resolutions = valid_resolutions[valid_resolutions['RESOLVED TIME'] >= valid_resolutions['EVENT DAY']]

    # Debug
    print("All months in dataset:", df['month'].value_counts().to_dict())
    print("Months in valid_resolutions:", valid_resolutions['month'].value_counts().to_dict())
    print("Total alarms per equipment (full_data):", df['SOURCE NAME'].value_counts().to_dict())
    print("Total alarms per equipment (valid_resolutions):", valid_resolutions['SOURCE NAME'].value_counts().to_dict())
    print("Total alarms per condition (full_data):", df['NAME'].value_counts().to_dict())

    return df, valid_resolutions

def analyze_equipment_source(data, full_data):
    """Analyze equipment sources, including alarm counts, resolution times, and frequency"""
    # Per-condition resolution times
    condition_times = data[data['resolution_minutes'].notna()].groupby(['SOURCE NAME', 'NAME'])['resolution_minutes'].mean().reset_index()
    condition_times_dict = condition_times.groupby('SOURCE NAME', group_keys=False).apply(
        lambda x: dict(zip(x['NAME'], x['resolution_minutes'].round(2))), include_groups=False
    ).to_dict()

    # Main aggregation
    name_count = full_data.groupby('SOURCE NAME').agg({
        'NAME': lambda x: dict(x.value_counts()),
        'PATH': 'first',
        'month': lambda x: sorted(x.unique(), key=lambda m: pd.to_datetime(m, format='%B').month),
        'week_of_month': lambda x: [f"Week {w}" for w in sorted(x.unique())],
        'day': 'nunique',
        'week': 'nunique',
        'resolution_minutes': [
            'size',
            lambda x: x.isna().sum(),
            lambda x: x.notna().sum(),
            'mean'
        ]
    }).reset_index()

    # Flatten columns
    name_count.columns = ['Source name', 'Name', 'Path', 'Month', 'Week of Month', 'Unique Days', 'Unique Weeks',
                          'Total Count', 'Unresolved', 'Resolved', 'Total Resolution Time']


    # Calculate frequency
    name_count['Frequency'] = name_count.apply(
        lambda row: {
            'Day': round(row['Total Count'] / row.get('Unique Days', 1), 2) if row.get('Unique Days', 1) > 0 else 0,
            'Week': round(row['Total Count'] / row.get('Unique Weeks', 1), 2) if row.get('Unique Weeks', 1) > 0 else 0,
            'Month': round(row['Total Count'] / len(row.get('Month', [])), 2) if len(row.get('Month', [])) > 0 else 0
        },
        axis=1
    )

    # Time taken to resolve
    name_count['Time taken to resolve in minutes'] = name_count.apply(
        lambda row: {
            'Total': round(row['Total Resolution Time'], 2) if pd.notna(row['Total Resolution Time']) else None,
            'By Condition': condition_times_dict.get(row['Source name'], {})
        },
        axis=1
    )

    # Drop temporary columns
    name_count = name_count.drop(columns=['Total Resolution Time', 'Unique Days', 'Unique Weeks'])

    # Format Name
    name_count['Name'] = name_count['Name'].apply(lambda x: [f"{k}: {v}" for k, v in x.items()])
    name_count = name_count.sort_values('Total Count', ascending=False)


    return name_count.to_dict('records')

def analyze_specific_equipment(data, source_name):
    """Analyze a specific equipment by SOURCE NAME"""
    source_name = next((s for s in data['SOURCE NAME'].unique() if s.lower() == source_name.lower()), source_name)
    filtered_data = data[data['SOURCE NAME'] == source_name]
    if filtered_data.empty:
        return f"No data available for equipment: {source_name}"

    # Per-condition resolution times
    condition_times = filtered_data[filtered_data['resolution_minutes'].notna()].groupby(['SOURCE NAME', 'NAME'])['resolution_minutes'].mean().reset_index()
    condition_times_dict = condition_times.groupby('SOURCE NAME', group_keys=False).apply(
        lambda x: dict(zip(x['NAME'], x['resolution_minutes'].round(2))), include_groups=False
    ).to_dict()

    # Main aggregation
    results = filtered_data.groupby('SOURCE NAME').agg({
        'NAME': lambda x: dict(x.value_counts()),
        'PATH': 'first',
        'month': lambda x: sorted(x.unique(), key=lambda m: pd.to_datetime(m, format='%B').month),
        'week_of_month': lambda x: [f"Week {w}" for w in sorted(x.unique())],
        'day': 'nunique',
        'week': 'nunique',
        'resolution_minutes': [
            'size',
            lambda x: x.isna().sum(),
            lambda x: x.notna().sum(),
            'mean'
        ]
    }).reset_index()

    # Flatten columns
    results.columns = ['Source name', 'Name', 'Path', 'Month', 'Week of Month', 'Unique Days', 'Unique Weeks',
                       'Total Count', 'Unresolved', 'Resolved', 'Total Resolution Time']


    # Calculate frequency
    results['Frequency'] = results.apply(
        lambda row: {
            'Day': round(row['Total Count'] / row.get('Unique Days', 1), 2) if row.get('Unique Days', 1) > 0 else 0,
            'Week': round(row['Total Count'] / row.get('Unique Weeks', 1), 2) if row.get('Unique Weeks', 1) > 0 else 0,
            'Month': round(row['Total Count'] / len(row.get('Month', [])), 2) if len(row.get('Month', [])) > 0 else 0
        },
        axis=1
    )

    # Time taken to resolve
    results['Time taken to resolve in minutes'] = results.apply(
        lambda row: {
            'Total': round(row['Total Resolution Time'], 2) if pd.notna(row['Total Resolution Time']) else None,
            'By Condition': condition_times_dict.get(row['Source name'], {})
        },
        axis=1
    )

    # Drop temporary columns
    results = results.drop(columns=['Total Resolution Time', 'Unique Days', 'Unique Weeks'])

    # Format Name
    results['Name'] = results['Name'].apply(lambda x: [f"{k}: {v}" for k, v in x.items()])
    results = results.sort_values('Total Count', ascending=False)



    return results.to_dict('records')

def analyze_condition_name(data, full_data):
    """Analyze all alarm conditions dynamically with frequency"""
    results = {}
    conditions = full_data['NAME'].unique()

    # Aggregate by condition and source name
    cond_freq = full_data.groupby(['NAME', 'SOURCE NAME']).agg({
        'PATH': 'first',
        'resolution_minutes': [
            'count',
            'mean'
        ],
        'day': 'nunique',
        'week': 'nunique',
        'month': lambda x: list(x.unique())
    }).reset_index()

    # Flatten columns
    cond_freq.columns = [
        'Condition name', 'Source name', 'Path', 'count', 'time taken',
        'Unique Days', 'Unique Weeks', 'Months'
    ]

    # Calculate frequency
    cond_freq['Frequency'] = cond_freq.apply(
        lambda row: {
            'Day': round(row['count'] / row.get('Unique Days', 1), 2) if row.get('Unique Days', 1) > 0 else 0,
            'Week': round(row['count'] / row.get('Unique Weeks', 1), 2) if row.get('Unique Weeks', 1) > 0 else 0,
            'Month': round(row['count'] / len(row.get('Months', [])), 2) if len(row.get('Months', [])) > 0 else 0
        },
        axis=1
    )

    # Format output to match desired structure
    for cond in conditions:
        cond_key = cond.lower().replace(' ', '_')
        cond_data = cond_freq[cond_freq['Condition name'] == cond]
        if not cond_data.empty:
            # Select and reorder columns: Source name, Path, count, time taken, Frequency
            result = cond_data[['Source name', 'Path', 'count', 'time taken', 'Frequency']].to_dict('records')
            results[cond_key] = result
        else:
            results[cond_key] = f"No data available for condition: {cond}"

    return results

def analyze_specific_condition(data, full_data, condition):
    """Analyze a specific condition with frequency"""
    condition = next((c for c in full_data['NAME'].unique() if c.lower() == condition.lower()), condition)
    cond_data = full_data[full_data['NAME'] == condition]
    if cond_data.empty:
        return f"No data available for condition: {condition}"

    # Perform aggregation
    cond_results = cond_data.groupby(['NAME', 'SOURCE NAME']).agg({
        'PATH': 'first',
        'resolution_minutes': [
            'count',
            'mean'
        ],
        'day': 'nunique',
        'week': 'nunique',
        'month': lambda x: list(x.unique())
    }).reset_index()

    # Flatten columns
    cond_results.columns = [
        'Condition name', 'Source name', 'Path', 'count', 'time taken',
        'Unique Days', 'Unique Weeks', 'Months'
    ]

    # Calculate frequency
    cond_results['Frequency'] = cond_results.apply(
        lambda row: {
            'Day': round(row['count'] / row.get('Unique Days', 1), 2) if row.get('Unique Days', 1) > 0 else 0,
            'Week': round(row['count'] / row.get('Unique Weeks', 1), 2) if row.get('Unique Weeks', 1) > 0 else 0,
            'Month': round(row['count'] / len(row.get('Months', [])), 2) if len(row.get('Months', [])) > 0 else 0
        },
        axis=1
    )

    # Format output to match desired structure
    result = cond_results[['Source name', 'Path', 'count', 'time taken', 'Frequency']].to_dict('records')


    return result

def save_and_download(data, filename_prefix):
    """Save analysis results to JSON and download"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{filename_prefix}_{timestamp}.json"
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2, cls=CustomJSONEncoder)
    files.download(filename)
    print(f"✅ Downloaded: {filename}")

def main():
    df, valid_resolutions = load_and_clean_data()
    if df is None or valid_resolutions is None:
        print("Exiting due to data loading error.")
        return

    equipment_results = analyze_equipment_source(valid_resolutions, df)
    condition_results = analyze_condition_name(valid_resolutions, df)

    while True:
        print("\n🔍 MAIN MENU")
        print("1. Alarm Equipment Analysis")
        print("2. Alarm Condition Name Analysis")
        print("3. Export Full Results")
        print("4. Exit")

        main_choice = input("Select option: ")

        if main_choice == "1":
            while True:
                print("\nALARM EQUIPMENT ANALYSIS")
                print("1. All equipment analysis")
                print("2. Type the specific equipment to analyze")
                print("3. Exit")

                equip_choice = input("Select analysis: ")

                if equip_choice == "1":
                    display(Markdown("## All Equipment Analysis"))
                    display(pd.DataFrame(equipment_results))
                    save_and_download(equipment_results, "Equipment_All_Analysis")

                elif equip_choice == "2":
                    print("Available equipment:", ", ".join(df['SOURCE NAME'].unique()))
                    source_name = input("Enter equipment name (e.g., KFC Abdul Nasser New - 112750): ")
                    specific_results = analyze_specific_equipment(df, source_name)
                    if isinstance(specific_results, str):
                        print(specific_results)
                    else:
                        display(Markdown(f"## Analysis for Equipment: {source_name}"))
                        display(pd.DataFrame(specific_results))
                        save_and_download(specific_results, f"Equipment_{source_name.replace(' ', '_')}_Analysis")

                elif equip_choice == "3":
                    break
                else:
                    print("Invalid selection. Please choose 1, 2, or 3.")

        elif main_choice == "2":
            while True:
                print("\nALARM CONDITION NAME ANALYSIS")
                print("1. All alarm name condition analysis")
                print("2. Type available condition for analysis")
                print("3. Exit")

                cond_choice = input("Select condition: ")

                if cond_choice == "1":
                    display(Markdown("## All Alarm Condition Analysis"))
                    for cond, result in condition_results.items():
                        display(Markdown(f"### {cond.replace('_', ' ').title()}"))
                        if isinstance(result, str):
                            print(result)
                        else:
                            display(pd.DataFrame(result))
                    save_and_download(condition_results, "Condition_All_Analysis")

                elif cond_choice == "2":
                    print("Available conditions:", ", ".join(df['NAME'].unique()))
                    condition = input("Enter condition name (e.g., Site Not Communicating, Door Open): ")
                    specific_results = analyze_specific_condition(valid_resolutions, df, condition)
                    if isinstance(specific_results, str):
                        print(specific_results)
                    else:
                        display(Markdown(f"## Analysis for Condition: {condition}"))
                        display(pd.DataFrame(specific_results))
                        save_and_download(specific_results, f"Condition_{condition.replace(' ', '_')}_Analysis")

                elif cond_choice == "3":
                    break
                else:
                    print("Invalid selection. Please choose 1, 2, or 3.")

        elif main_choice == "3":
            full_results = {
                "Equipment_Analysis": equipment_results,
                "Condition_Analysis": condition_results
            }
            save_and_download(full_results, "Full_Analysis_Results")

        elif main_choice == "4":
            print("Exiting program...")
            break
        else:
            print("Invalid selection. Please choose 1, 2, 3, or 4.")

if __name__ == "__main__":
    main()

Please upload your Excel file:


Saving week.xlsx to week.xlsx
All months in dataset: {'May': 6}
Months in valid_resolutions: {'May': 6}
Total alarms per equipment (full_data): {'H 5514144 EM 01': 6}
Total alarms per equipment (valid_resolutions): {'H 5514144 EM 01': 6}
Total alarms per condition (full_data): {'No power': 6}

🔍 MAIN MENU
1. Alarm Equipment Analysis
2. Alarm Condition Name Analysis
3. Export Full Results
4. Exit
Select option: 1

ALARM EQUIPMENT ANALYSIS
1. All equipment analysis
2. Type the specific equipment to analyze
3. Exit
Select analysis: 1


## All Equipment Analysis

,Source name,Name,Path,Month,Week of Month,Total Count,Unresolved,Resolved,Frequency,Time taken to resolve in minutes
0,H 5514144 EM 01,[No power: 6],KSA/KSA Central/Hardees Imam Saud 2 - 5514144,[May],"[Week 1, Week 2, Week 3, Week 4, Week 5]",6,0,6,"{'Day': 1.0, 'Week': 1.2, 'Month': 6.0}","{'Total': 0.0, 'By Condition': {'No power': 0.0}}"


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Downloaded: Equipment_All_Analysis_20250604_065328.json

ALARM EQUIPMENT ANALYSIS
1. All equipment analysis
2. Type the specific equipment to analyze
3. Exit
Select analysis: 3

🔍 MAIN MENU
1. Alarm Equipment Analysis
2. Alarm Condition Name Analysis
3. Export Full Results
4. Exit
Select option: 4
Exiting program...
